In [115]:
from mypython.MLmodel import * 
from mypython.Linearmodel import *
from mypython.Treemodel import *

import numpy as np
import matplotlib.pyplot as plt
import cuml
import sklearn
import lightgbm
import xgboost

x = np.load('/home/qianshuofu/factor_qianshuofu/Data/data_feature.npy')
y = np.load("/home/qianshuofu/factor_qianshuofu/Data/data_label.npy")
# y = np.load("/home/qianshuofu/factor_qianshuofu/Data/data_label_scale.npy")

# model_linear = MLmodel(x,y,MyLinear_Gpu(),name='linear')
# model_lasso = MLmodel(x,y,MyLasso_Gpu(),name='lasso')
# model_ridge = MLmodel(x,y,MyRidge_Gpu(),name='ridge')
# model_ElasticNet = MLmodel(x,y,MyElasticNet_Gpu(),name='elasticnet')

# model_DT = MLmodel(x,y,MyDTRegressor(),name='DT')
# model_RF = MLmodel(x,y,MyRFRegressor_Gpu(),name='RF')
# model_LGBM = MLmodel(x,y,MyLGBMRegressor(),name='LGBM')
# model_XGB = MLmodel(x,y,MyXGBRegressor(),name='XGB')

# def main():
#     model_dict = {'linear': MyLinear_Gpu(),
#                 'lasso': MyLasso_Gpu(),
#                 'ridge': MyRidge_Gpu(),
#                 'elasticnet': MyElasticNet_Gpu(),
#                 # 'DT': MyDTRegressor(),     # 太慢了跑不出来
#                 'RF': MyRFRegressor_Gpu(),  # gpu加速也很慢  7min
#                 'LGBM': MyLGBMRegressor(),  # 不使用gpu时,lgbm原生接口和lightgbm.sklearn接口速度一致,45s
#                 'XGB': MyXGBRegressor() }   # 不使用gpu时,xgboost接口和xgboost.sklearn接口速度一致,6min40s,慢死了...

#     for name,model in model_dict.items():
#         MLmodel(x,y,model,name).split_train_test(random_state=42).fit_model().predict().evaluate_model()

# main()

In [119]:
x_train, x_test,  y_train, y_test  = sklearn.model_selection.train_test_split(x,y,test_size=0.3,random_state=42)
x_train, x_valid, y_train, y_valid = sklearn.model_selection.train_test_split(x_train,y_train,test_size=0.1,random_state=42)

# model = lightgbm.train({},lightgbm.Dataset(x_train,y_train),valid_sets=lightgbm.Dataset(x_train,y_train),verbose_eval=1)
# model = lightgbm.train({},lightgbm.Dataset(x_train,y_train),verbose_eval=1)
model = lightgbm.sklearn.LGBMRegressor(n_estimators=500).fit(x_train,y_train,eval_set=[(x_valid,y_valid)],early_stopping_rounds=50, eval_metric=['l2','l1'])

y_pred = model.predict(x_test)
print(sklearn.metrics.mean_squared_error(y_test,y_pred))
print(np.corrcoef(y_pred.argsort().argsort(),y_test.argsort().argsort()))

# without scaling: l2: 0.000576, Corr:0.118, std: 0.024**2=0.000576
# scaling:         l2: 0.96627,  Corr:0.122, std: 1**2

/home/qianshuofu/anaconda3/envs/rapids-22.12/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.000635311	valid_0's l1: 0.0147704
[2]	valid_0's l2: 0.000634422	valid_0's l1: 0.0147609
[3]	valid_0's l2: 0.0006337	valid_0's l1: 0.0147542
[4]	valid_0's l2: 0.000632986	valid_0's l1: 0.014748
[5]	valid_0's l2: 0.000632434	valid_0's l1: 0.0147443
[6]	valid_0's l2: 0.00063193	valid_0's l1: 0.0147399
[7]	valid_0's l2: 0.000631498	valid_0's l1: 0.0147357
[8]	valid_0's l2: 0.000631177	valid_0's l1: 0.0147309
[9]	valid_0's l2: 0.000630724	valid_0's l1: 0.0147269
[10]	valid_0's l2: 0.000630443	valid_0's l1: 0.0147245
[11]	valid_0's l2: 0.000630084	valid_0's l1: 0.014721
[12]	valid_0's l2: 0.00062987	valid_0's l1: 0.0147185
[13]	valid_0's l2: 0.000629499	valid_0's l1: 0.0147156
[14]	valid_0's l2: 0.00062926	valid_0's l1: 0.0147147
[15]	valid_0's l2: 0.000629049	valid_0's l1: 0.0147134
[16]	valid_0's l2: 0.000628757	valid_0's l1: 0.014711
[17]	valid_0's l2: 0.000628631	valid_0's l1: 0.0147098
[18]	valid_0's l2: 0.000628386	valid_0's l1: 0.0147087
[19]	valid_0's l2: 0.000628

In [117]:
y_pred

array([-0.00124641,  0.00086417,  0.0006297 , ..., -0.00061758,
       -0.00107229,  0.00165104])

In [118]:
help(lightgbm.sklearn.LGBMRegressor)

Help on class LGBMRegressor in module lightgbm.sklearn:

class LGBMRegressor(sklearn.base.RegressorMixin, LGBMModel)
 |  LGBMRegressor(boosting_type: str = 'gbdt', num_leaves: int = 31, max_depth: int = -1, learning_rate: float = 0.1, n_estimators: int = 100, subsample_for_bin: int = 200000, objective: Union[str, Callable, NoneType] = None, class_weight: Union[Dict, str, NoneType] = None, min_split_gain: float = 0.0, min_child_weight: float = 0.001, min_child_samples: int = 20, subsample: float = 1.0, subsample_freq: int = 0, colsample_bytree: float = 1.0, reg_alpha: float = 0.0, reg_lambda: float = 0.0, random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None, n_jobs: int = -1, silent: Union[bool, str] = 'warn', importance_type: str = 'split', **kwargs)
 |  
 |  LightGBM regressor.
 |  
 |  Method resolution order:
 |      LGBMRegressor
 |      sklearn.base.RegressorMixin
 |      LGBMModel
 |      sklearn.base.BaseEstimator
 |      builtins.object
 |  
 |  Methods de